<a href="https://colab.research.google.com/github/johnnyd212/comp551-assignment-2/blob/main/analysis%201-2-3-early-stopping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numpy pandas matplotlib sys pathlib itertools warnings imp

ERROR: Could not find a version that satisfies the requirement sys (from versions: none)
ERROR: No matching distribution found for sys


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
from pathlib import Path
sys.path.append(str(Path().resolve().parent / 'src'))



## Task 0: Pre-processing

### Loading the data

Our CSV file is unlabeled, so we create our column names following the description in spambase.DOCUMENTATION:

In [3]:
import numpy as np
import itertools

# let y^(i)_hat \in [0, 1] be our prediction (a probability)
# that x^{(i)} is a yes-instance of the class.

# For our logistic classifier, y^{(i)}_hat := \sigma(w^T x^{(i)})

# The likelihood of our model classifying x^(i) correctly is
# the Bernoulli(y^{(i)} ; y^(i)_hat)
# = (\hat{y}^{(i)})^{y^{(i)}} * (1 - \hat{y}^{(i)})^{1 - y^{(i)}}

# Thus the negative log likelihood is then
# - y^{(i)} \log(\hat{y}^{(i)}) + (1 - y^{(i)})\log(1 - \hat{y}^{(i)})

# This is the objective function of our logistic classifier.
def cross_entropy_loss(pred, y):
  eps = 1e-15
  pred = np.clip(pred, eps, 1 - eps)  # prevent log(0)
  return (-y * np.log(pred) - (1 - y) * np.log(1 - pred)).sum()


# A numerically stable implementation of the sigmoid function
# using the "logsumexp" trick
def sigmoid(z):
    # Set c to be the elementwise max of 0 and -z
    c = np.maximum(0, -z)

    try:
        return np.exp(-c - np.log(np.exp(-c) + np.exp(-z - c)))
    except RuntimeWarning:
            print('Exponential overflow in sigmoid')


class LogisticRegressor:
    def __init__(self,
                 batch_size=1,
                 learning_rate=0.0005,
                 num_epochs=100,
                 regularization_strength=0,
                 early_stopping = False,
                 seed=None):

        self.batch_size = batch_size
        self.learning_rate=learning_rate
        self.num_epochs=num_epochs
        self.regularization_strength = regularization_strength
        self.early_stopping = early_stopping

        self.rng = np.random.default_rng(seed)

    # TODO: implement ADAM
    # TODO: Is it possible to get a fully vectorized implementation
    #       that eliminates the need for the outerloop?
    # TODO: clip predictions to prevent log(0), clip gradients for numerical stability
    def fit(self, X, y, learning_curve=False, add_bias=True):
        if add_bias:
            # Concatenate 1's for bias terms
            X = np.concatenate([np.ones((X.shape[0], 1)), X], axis=1)

        self.weights = self.rng.random((X.shape[1], 1))

        #if(learning_curve): REMOVE, otherwise it doesn't work
        errors = []

        epochs_since_improvement = 0
        min_loss = float('inf')
        epoch_count = 0

        while(epoch_count < self.num_epochs and epochs_since_improvement < 10):
            # Permutation to apply to our rows for stochasticity
            indices = self.rng.permutation(X.shape[0])
            epoch_count = epoch_count + 1

            for batch in itertools.batched(indices, self.batch_size):
                # Convert to ndarray to trigger advanced indexing
                ndbatch = np.asarray(batch)
                # Weight update
                try:
                    self.weights -= self.learning_rate * self.loss_gradient(X[ndbatch], y[ndbatch])
                except TypeError:
                    print('Prediction error')
            if (self.early_stopping):
                loss = self.loss(X, y)
                if loss < min_loss:
                    min_loss = loss
                else:
                    epochs_since_improvement += 1


            if(learning_curve):
                try:
                    errors.append(self.loss(X, y))
                except RuntimeWarning:
                    print('Division by 0 in log')

        return errors



    # Our loss is the L2-regularized cross entropy loss
    def loss(self, X, y):
        ce_loss = cross_entropy_loss(self.predict(X), y)
        # Exclude bias term from regularization penalty
        reg_penalty = (self.regularization_strength / 2) * np.sum(self.weights[1:] ** 2)
        return ce_loss + reg_penalty


  #For data analysis, it is the sum of True Positives () and True Negatives () divided by the total sample size ().

    # The gradient of our loss function
    def loss_gradient(self, X, y):
        grad = X.T @ (self.predict(X) - y)
        # Add derivative term for regularization
        grad[1:] += self.regularization_strength * self.weights[1:]
        return grad

    # prediction of logistic classifer use the sigmoid function
    def predict(self, X, add_bias = False):
        # If X doesn't have the bias column (i.e., its number of columns is one less than the number of weights), add a column of ones.
        if X.shape[1] == self.weights.shape[0] - 1:
            X_with_bias = np.concatenate([np.ones((X.shape[0], 1)), X], axis=1)
            return sigmoid(X_with_bias @ self.weights)
        elif X.shape[1] == self.weights.shape[0]:
            # X already has the bias column (or model was fitted without bias)
            return sigmoid(X @ self.weights)
        else:
            raise ValueError("Input X dimensions do not match model's weights.")

    #Score Accuracy
    def accuracy(self, X, y):
        good_predictions = ((self.predict(X) > 0.5) == y)
        return np.mean(good_predictions)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (2068833110.py, line 41)

In [ ]:
word_freq_names = [f'word_freq_{i}' for i in range(1, 49)]
char_freq_names = [f'char_freq_{i}' for i in range(1, 7)]
cap_run_features = ['capital_run_length_' + s for s in ['average', 'longest', 'total']]

col_names = (
    word_freq_names +
    char_freq_names +
    cap_run_features +
    ['spam']
)

In [ ]:
df = pd.read_csv('spambase.data', header=None, names=col_names)
df.head()

### Feature normalization

We can see that the ranges of the values for our integral features are quite large relative to our word and char frequencies:

In [ ]:
df[cap_run_features].describe()

I tried to train the regressor without normalizing at first, but the sigmoid function saturated, and we were getting numerical issues of taking logs of 0 in our cross entropy loss as a consequence. To address this, we use min-max normalization to make our integral count features in line with our 0-1 word and char frequency features.

In [ ]:
cap_minima = df[cap_run_features].min()
cap_maxima = df[cap_run_features].max()

df[cap_run_features] =  ( (df[cap_run_features] - cap_minima) /
                         (cap_maxima - cap_minima) )

In [ ]:
df[cap_run_features].describe()

### Random Seed

For reproducibility, we'll set the seed for the random generators we use (e.g. for train-test spliiting, stochastic gradient descent, etc...).

In [ ]:
rng_seed = 551
rng = np.random.default_rng(seed=rng_seed)

### Train-test split

We'll implement a train-test split of 5-95, that is, we use 5% of the data for training, and hold out the remaining 95% for testing. (In the interest of implementing our classifier from 'scratch', we don't use train_test_split from scikit-learn.)

In [ ]:
train_ratio = 0.05
N = len(df)

# Train-test split
df_perm = df.iloc[rng.permutation(N)]
split_index = int(N * train_ratio)
df_train = df_perm.iloc[:split_index]
df_test = df_perm.iloc[split_index:]

X_train = df_train.drop(columns=['spam'])
y_train = df_train['spam']
X_test = df_test.drop(columns=['spam'])
y_test = df_test['spam']

## Task 1: Logistic Regression with SGD

We'll initialize two logistic regressors, one with no regularization and the other with regularization strength 0.001, to compare their loss curves. We'll initialize the remaining hyperparameters to the following values:
- batch_size = 1
- learning_rate = 0.01
- num_epochs = 100

In [ ]:
reg_str = 0.001

In [ ]:
model_no_reg = LogisticRegressor(seed=551)
model_reg = LogisticRegressor(regularization_strength=reg_str, seed=551)

learning_curve_no_reg = model_no_reg.fit(X_train.to_numpy(), y_train.to_numpy(),
                                         learning_curve=True)
learning_curve_reg = model_reg.fit(X_train.to_numpy(), y_train.to_numpy(),
                                   learning_curve=True)

In [ ]:
plt.plot(np.arange(1, model_no_reg.num_epochs + 1),
            learning_curve_no_reg,
            'r--', label='\u03BB = 0')
plt.plot(np.arange(1, model_reg.num_epochs + 1),
            learning_curve_reg,
            'b--', label=f'\u03BB = {reg_str}')

plt.xlabel("Epoch")
plt.ylabel("Cost")
plt.title("Learning Curves")
plt.legend()

# **Part 2: Hyperparameter Tuning with K-Fold Cross Validation**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from random import randrange
from sklearn.model_selection import ParameterGrid
import itertools # Added for completeness as it's used in LogisticRegressor

def knuth_shuffle(x):
    for i in range(len(x)-1, 0, -1):
        j = randrange(i + 1)
        x[i], x[j] = x[j], x[i]
    return x

def k_fold_validation(train_x, train_y, model_class, params, n_folds=5):
  '''
  This function performs k-fold validation and returns the average validation loss
  '''

  if n_folds is None:
    n_folds = train_x.shape[0]
  # Ensure n_folds is not greater than the number of samples in X_data
  if n_folds > train_x.shape[0]:
      print(f"Warning: n_folds ({n_folds}) is greater than number of samples ({train_x.shape[0]}). Setting n_folds to number of samples.")
      n_folds = train_x.shape[0]

  # K-fold Validation
  # Convert to numpy arrays before splitting to avoid iloc/index issues with potentially modified index after splitting
  X_data_np = train_x.to_numpy()
  y_data_np = train_y.to_numpy().reshape(-1, 1) # Reshape y to (N, 1) right away for consistency

  indices = list(range(X_data_np.shape[0]))
  knuth_shuffle(indices)
  folds = np.array_split(indices, n_folds)

  d_features = X_data_np.shape[1]
  validation_loss_array = np.zeros(n_folds)
  validation_accuracy_array = np.zeros(n_folds)
  train_loss_array = np.zeros(n_folds)
  train_accuracy_array = np.zeros(n_folds)

  for i in range(n_folds):
    # Identify validation set indices and training set indices for the current fold
    validation_index = folds[i]
    training_index = np.concatenate([f for j, f in enumerate(folds) if j != i])

    # Split data using the numpy arrays
    validation_x_fold = X_data_np[validation_index]
    train_x_fold = X_data_np[training_index]

    validation_y_fold = y_data_np[validation_index]
    train_y_fold = y_data_np[training_index]

    model = model_class(**params)

    # This portion might be worth modifying if we use a model that uses gradient instead
    model.fit(train_x_fold, train_y_fold) # train_y_fold is already (N_fold, 1)

    # Calculate loss and accuracy - pass features (X) and true labels (y)
    # The model's loss and accuracy methods will call model.predict(X) internally
    validation_loss_array[i] = model.loss(validation_x_fold, validation_y_fold)
    train_loss_array[i] = model.loss(train_x_fold, train_y_fold)

    validation_accuracy_array[i] = model.accuracy(validation_x_fold, validation_y_fold)
    train_accuracy_array[i] = model.accuracy(train_x_fold, train_y_fold)

  avg_val_loss = np.mean(validation_loss_array)
  std_val_loss = np.std(validation_loss_array)
  avg_val_accuracy = np.mean(validation_accuracy_array)

  avg_train_loss = np.mean(train_loss_array)
  std_train_loss = np.std(train_loss_array)
  avg_train_accuracy = np.mean(train_accuracy_array)

  return d_features, avg_val_loss, std_val_loss, avg_val_accuracy, avg_train_loss, std_train_loss, avg_train_accuracy


def model_selection (model_classes, param_grid, train_x, train_y, n_folds = None):
  '''
  iterates through each combination of hyperparameter
  '''
  if n_folds is None:
    n_folds = train_x.shape[0]

  fold_losses = []
  fold_params = []
  all_d_features = []
  std_vals = []

  for model_class in model_classes:
    print("Model_Class: " + str(model_class))

    for params in ParameterGrid(param_grid):
      d_features, avg_val_loss, std_val_loss, avg_val_accuracy, _ , _, _ = k_fold_validation(train_x, train_y, model_class, params, n_folds = n_folds)
      fold_losses.append(avg_val_loss)
      fold_params.append(params)
      all_d_features.append(d_features)
      std_vals.append(std_val_loss)

  # Convert to numpy arrays for easier indexing and calculations
  fold_losses = np.array(fold_losses)
  all_d_features = np.array(all_d_features)

  # Find lambda with minimum validation loss
  min_index = np.argmin(val_fold_losses)
  min_loss = val_fold_losses[min_index]
  min_loss_std = std_vals[min_index]

  threshold = min_loss + min_loss_std

  loss_candidates = np.where(fold_losses <= (threshold))[0]

  fold_losses = np.array(fold_losses)
  fold_params = np.array(fold_params)
  all_d_features = np.array(all_d_features)

  all_candidates = zip(all_d_features, fold_losses, std_vals, fold_params)


  cand_features = all_d_features[loss_candidates]
  cand_losses = fold_losses[loss_candidates]
  cand_params = [fold_params[i] for i in loss_candidates]

  cands = zip(cand_features, cand_losses, cand_params)
  sorted_cands = sorted(cands, key=lambda x:(x[0], x[1]))

  best_model = sorted_cands[0]
  print ("The best model is "+ str(best_model[2]) + ". It has " + (str(best_model[0]) + " features, and produces a loss of " + str(best_model[1])))

  return best_model, all_candidates


param_grid = {
  "batch_size": [1, 16, 64],
  "learning_rate": [1, 0.1, 0.01, 0.001, 0.0001],
  "num_epochs": [50, 100, 200],
  "regularization_strength": [0.001]
}


model_classes = [LogisticRegressor]
best_model, all_results = model_selection(model_classes, param_grid, X_train, y_train, n_folds = 100)


print(all_results)


# **Part 3: Bias-Variance Trade-Off via λ Sweep**

In [ ]:
import numpy as np

# Generate values from 1e-6 to 1 on a log scale
regularization_strength = np.logspace(-6, 0, num=25)  # 25 values from 1e-6 to 1

# Optionally, include 0 explicitly if your model supports it
regularization_strength = np.insert(regularization_strength, 0, 0)



param_grid = {
    "batch_size": [16],
    "learning_rate": [0.1],
    "num_epochs": [200],
    "regularization_strength": regularization_strength
}


def lambda_sweep (model_classes, param_grid, train_x, train_y, n_folds = 100):
  '''
  iterates through each combination of hyperparameter
  '''
  shape = [len(v) for v in list(param_grid.values())]
  d_features_grid = np.zeros(shape)
  val_loss_grid = np.zeros(shape)

  lambda_array = []

  train_fold_losses = []
  train_fold_accuracy = []
  train_fold_params = []

  val_fold_losses = []
  val_fold_accuracy = []
  val_fold_params = []
  all_d_features = []
  std_vals = []




  for model_class in model_classes:

    for params in ParameterGrid(param_grid):
      model = model_class(**params)
      d_features, avg_val_loss, std_val_loss, avg_val_accuracy, avg_train_loss, std_train_loss, avg_train_accuracy = k_fold_validation(train_x, train_y, model_class, params, n_folds)

      #Append Losses, Parameters, and Accuracy for validation set
      val_fold_losses.append(avg_val_loss)
      val_fold_accuracy.append(avg_val_accuracy)
      val_fold_params.append(params)

      #Append Losses, Parameters, and Accuracy for test set
      train_fold_losses.append(avg_train_loss)
      train_fold_accuracy.append(avg_train_accuracy)
      train_fold_params.append(params)
      std_vals.append(std_val_loss)


      lambda_array.append(params["regularization_strength"])

      all_d_features.append(d_features)

      #d_features_grid.append(d_features)


  #Graph relationship between log lambda and training/validation loss
  plt.figure(figsize=(10, 5))
  plt.plot(lambda_array, val_fold_losses, marker = 'o')
  plt.plot(lambda_array, train_fold_losses, marker = 'o')
  plt.xscale('log')
  plt.legend(["Validation Loss", "Training Loss"])
  plt.xlabel("Log Lambda")
  plt.ylabel("Loss")
  plt.title("Loss vs Lambda")
  plt.show()


  #Graph relationship between log lambda and training/validation accuracy
  plt.figure(figsize=(10, 5))
  plt.plot(lambda_array, val_fold_accuracy, marker = 'o')
  plt.plot(lambda_array, train_fold_accuracy, marker = 'o')
  plt.xscale('log')
  plt.legend(["Validation Accuracy", "Training Accuracy"])
  plt.xlabel("Log Lambda")
  plt.ylabel("Accuracy")
  plt.title("Accuracy vs Lambda")
  plt.show()


  #Model Selection based on standard deviation + model complexity
  std_val_loss = np.std(val_fold_losses)
  min_val_loss = np.min(val_fold_losses)

  min_index = np.argmin(val_fold_losses)
  min_loss = val_fold_losses[min_index]
  min_loss_std = std_vals[min_index]

  threshold = min_loss + min_loss_std

  loss_candidates = np.where(val_fold_losses <= threshold)[0]


  fold_losses = np.array(val_fold_losses)
  fold_params = np.array(val_fold_params)
  all_d_features = np.array(all_d_features)

  all_candidates = zip(all_d_features, fold_losses, std_val_loss, fold_params)



  cand_features = all_d_features[loss_candidates]
  cand_losses = fold_losses[loss_candidates]
  cand_params = fold_params[loss_candidates]


  cands = zip(cand_features, cand_losses, cand_params)
  sorted_cands = sorted(cands, key=lambda x:(x[0], x[1]))

  best_model = sorted_cands[0]
  print ("The best model is "+ str(best_model[2]) + ". It has " + (str(best_model[0])) + "features, and produces a loss of " + (str(best_model[1])))

  return best_model, all_candidates


best_model, all_results = lambda_sweep(model_classes, param_grid, X_train, y_train)
print (all_results)





